In [1]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')
 
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_algorithms.utils import algorithm_globals
 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


import os 
import pandas as pd


2025-09-24 20:30:00.469096: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-24 20:30:00.478424: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 20:30:00.563673: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 20:30:00.630986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758724200.694951   16781 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758724200.70

ImportError: cannot import name 'BaseEstimator' from 'qiskit.primitives' (/home/silicon/opencv_learn/.venv/lib/python3.12/site-packages/qiskit/primitives/__init__.py)

In [ ]:
algorithm_globals.random_seed = 42
np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)

class FeatureExtractor(nn.Module):
    """Modified ResNet-inspired feature extractor for MNIST"""
    def __init__(self, input_channels=1, output_dim=8):
        super(FeatureExtractor, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Batch normalization
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        # Dropout
        self.dropout = nn.Dropout(0.5)

        # Dimension reduction layers (key improvement from the paper)
        self.conv_reduce = nn.Conv2d(128, 16, kernel_size=1)  # 1x1 conv for dimension reduction
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Final dimension reduction to match quantum circuit qubits
        self.fc_reduce = nn.Linear(16, output_dim)

    def forward(self, x):
        # Feature extraction
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))

        # Dimension reduction (paper's key contribution)
        x = torch.relu(self.conv_reduce(x))
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)

        # Final reduction to quantum circuit input size
        x = torch.tanh(self.fc_reduce(x))  # Tanh to keep values in [-1, 1] for angle embedding

        return x



#
class QuantumConvolutionalLayer:
    """Quantum Convolutional Layer as described in the paper"""
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits

    def create_circuit(self):
        qc = QuantumCircuit(self.num_qubits)

        # Parameterized gates for quantum convolution
        params = []
        for i in range(self.num_qubits - 1):
            # RY and RZ gates as shown in Figure 3 of the paper
            theta_y = Parameter(f'theta_y_{i}')
            theta_z = Parameter(f'theta_z_{i}')
            params.extend([theta_y, theta_z])

            qc.ry(theta_y, i)
            qc.rz(theta_z, i)
            qc.cx(i, i+1)

        return qc, params
#
class QuantumPoolingLayer:
    """Quantum Pooling Layer as described in the paper"""
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits

    def create_circuit(self):
        qc = QuantumCircuit(self.num_qubits)

        # Pooling operation - reduces qubits by half
        params = []
        for i in range(0, self.num_qubits-1, 2):
            theta_y = Parameter(f'pool_y_{i}')
            theta_z = Parameter(f'pool_z_{i}')
            params.extend([theta_y, theta_z])

            qc.ry(theta_y, i)
            qc.rz(theta_z, i)
            qc.cx(i, i+1)

        return qc, params

#
def create_quantum_neural_network(num_qubits=8):
    """Create the Quantum Neural Network (VQC) as used in the paper"""

    # Feature map for angle embedding
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)

    # Variational ansatz
    ansatz = RealAmplitudes(num_qubits, reps=3)

    # Create the complete quantum circuit
    qc = QuantumCircuit(num_qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # Observable for measurement
    observable = SparsePauliOp.from_list([("Z" + "I" * (num_qubits-1), 1.0)])

    # Create QNN
    qnn = EstimatorQNN(
        circuit=qc,
        observables=observable,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
    )

    return qnn


class HybridQuantumNeuralNetwork(nn.Module):
    """Complete Hybrid Classical-Quantum Neural Network"""
    def __init__(self, num_qubits=8, num_classes=10):
        super(HybridQuantumNeuralNetwork, self).__init__()

        # Classical feature extractor
        self.feature_extractor = FeatureExtractor(output_dim=num_qubits)

        # Quantum neural network
        self.qnn = create_quantum_neural_network(num_qubits)
        self.quantum_layer = TorchConnector(self.qnn)

        # Classical output layer
        self.classifier = nn.Linear(1, num_classes)

    def forward(self, x):
        # Extract features using classical network
        features = self.feature_extractor(x)

        # Process through quantum network
        quantum_output = self.quantum_layer(features)

        # Final classification
        output = self.classifier(quantum_output)

        return output


def load_and_preprocess_mnist():
    """Load and preprocess MNIST dataset"""
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Normalize pixel values
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Add channel dimension
    x_train = x_train.reshape(-1, 1, 28, 28)
    x_test = x_test.reshape(-1, 1, 28, 28)

    # Use a subset for faster training (as in the paper with 780 samples)
    # Sample 1000 training samples and 200 test samples
    indices = np.random.choice(len(x_train), len(x_train), replace=False)
    x_train_subset = x_train[indices]
    y_train_subset = y_train[indices]

    indices_test = np.random.choice(len(x_test), len(x_test), replace=False)
    x_test_subset = x_test[indices_test]
    y_test_subset = y_test[indices_test]

    return x_train_subset, y_train_subset, x_test_subset, y_test_subset


def calculate_metrics(y_true, y_pred):
    """Calculate performance metrics as in the paper"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }



def train_hybrid_model(no_of_epochs=5,learning_rate=0.001,graph_filename='ok.png',experiment_no=0):
    """Train the hybrid quantum neural network"""
    print("Loading and preprocessing MNIST dataset...")
    x_train, y_train, x_test, y_test = load_and_preprocess_mnist()

    # Convert to PyTorch tensors
    x_train_tensor = torch.FloatTensor(x_train)
    y_train_tensor = torch.LongTensor(y_train)
    x_test_tensor = torch.FloatTensor(x_test)
    y_test_tensor = torch.LongTensor(y_test)

    # Create data loaders
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    print("Creating hybrid quantum neural network...")
    model = HybridQuantumNeuralNetwork(num_qubits=8, num_classes=10)

    # Loss function and optimizer (using COBYLA equivalent - Adam for neural parts)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print("Training the model...")
    train_losses = []
    history = {
        "experiment_no":[],
        "epochs":[],
        # "batch":[],
        "train_loss": [],
        "train_accuracy": []
        }
    metric_history={
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': []
    }
    model.train()

    for epoch in range(no_of_epochs):
        epoch_loss = 0.0
        num_batches = 0
        correct, total = 0, 0

        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()

            try:
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                num_batches += 1
                predictions = torch.argmax(output, dim=1)
                correct += (predictions == target).sum().item()
                total += target.size(0)
                acc=(predictions == target).sum().item()/target.size(0)

                if batch_idx % 5 == 0:
                    print(f'Epoch {epoch+1}/{no_of_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}, Accuracy:{acc:.4f}')

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue

        avg_loss = epoch_loss / num_batches if num_batches > 0 else 0
        accuracy = correct / total if total > 0 else 0
        history["train_loss"].append(avg_loss)
        history["train_accuracy"].append(accuracy)
        history["experiment_no"].append(experiment_no)
        history['epochs'].append(epoch)
        # history['batch'].append('Epoch Level')

        train_losses.append(avg_loss)
        print(f'Epoch {epoch+1}/{no_of_epochs} completed. Average Loss: {avg_loss:.4f}')

    print("Evaluating the model...")
    model.eval()
    all_predictions = []
    all_targets = []
    # Accuracy, precision, recall, F1-score are computed once after training is done.
    with torch.no_grad():
        for data, target in test_loader:
            try:
                output = model(data)
                predictions = torch.argmax(output, dim=1)
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(target.cpu().numpy())
            except Exception as e:
                print(f"Error in evaluation: {e}")
                continue

    # Calculate metrics
    if all_predictions and all_targets:
        metrics = calculate_metrics(all_targets, all_predictions)
        metric_history['accuracy'].append(metrics['accuracy'])
        metric_history['precision'].append(metrics['precision'])
        metric_history['recall'].append(metrics['recall'])
        metric_history['f1_score'].append(metrics['f1_score'])
        print("\n" + "="*50)
        print("HYBRID QUANTUM NEURAL NETWORK RESULTS")
        print("="*50)
        print(f"Accuracy: {metrics['accuracy']:.3f}")
        print(f"Precision: {metrics['precision']:.3f}")
        print(f"Recall: {metrics['recall']:.3f}")
        print(f"F1-Score: {metrics['f1_score']:.3f}")
        print("="*50)

        # Plot training loss
        plt.figure(figsize=(10, 6))
        plt.plot(train_losses, 'b-', label='Training Loss')
        plt.title('Hybrid Quantum Neural Network - Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.savefig(graph_filename)
        plt.show()

        return model,metric_history, history
    else:
        print("No predictions were generated successfully.")
        return None, None,None



def demonstrate_quantum_circuits():
    """Demonstrate the quantum circuits used in the paper"""
    print("\nDemonstrating Quantum Circuit Components:")
    print("="*50)

    # Quantum Convolutional Layer
    print("1. Quantum Convolutional Layer (2-qubit example):")
    qconv = QuantumConvolutionalLayer(2)
    qc_conv, params_conv = qconv.create_circuit()
    print(qc_conv.draw())
    print(f"Parameters: {[p.name for p in params_conv]}")

    # Quantum Pooling Layer
    print("\n2. Quantum Pooling Layer (2-qubit example):")
    qpool = QuantumPoolingLayer(2)
    qc_pool, params_pool = qpool.create_circuit()
    print(qc_pool.draw())
    print(f"Parameters: {[p.name for p in params_pool]}")

    # Feature Map (Angle Embedding)
    print("\n3. Feature Map for Angle Embedding (4-qubit example):")
    feature_map = ZZFeatureMap(4, reps=1)
    print(feature_map.draw())


def main():
    """Main function to run the complete implementation"""
    print("Enhanced Hybrid Quantum Neural Network for Image Classification")
    print("Based on: 'Enhanced Hybrid Quantum Neural Network for Breast Cancer Detection'")
    print("Adapted for MNIST digit classification")
    print("\n")
    execution_count = get_ipython().execution_count
    cwd = os.getcwd()
    new_dir = os.path.join(cwd,f'experiment_{execution_count}')
    os.makedirs(new_dir,exist_ok=True)
    train_metrics_path = os.path.join(new_dir,'train_metrics.csv')
    test_metrics_path = os.path.join(new_dir,'test_metrics.csv')
    graph_filename = os.path.join(new_dir,'graph.png')
    # Demonstrate quantum circuits
    # demonstrate_quantum_circuits()val_accuracy

    # Train and evaluate the model
    print("\nStarting training process...")

    model, metrics,history = train_hybrid_model(no_of_epochs=10,graph_filename=graph_filename,experiment_no=execution_count)

    if model is not None:
        train=pd.DataFrame(history)
        train.to_csv(train_metrics_path)
        test=pd.DataFrame(metrics)
        test.to_csv(test_metrics_path)

        print("\nTraining completed successfully!")
        print("The model combines classical feature extraction with quantum neural networks")
        print("Key innovations from the paper:")
        print("- Convolutional layer for dimension reduction (instead of fully connected)")
        print("- Hybrid classical-quantum architecture")
        print("- Angle embedding for quantum feature encoding")
        print("- Variational Quantum Classifier (VQC)")
    else:
        print("Training failed. Please check the setup and try again.")


if __name__ == "__main__":
    # Check if required libraries are available
    try:
        main()
    except ImportError as e:
        print(f"Missing required library: {e}")
        print("Please install required packages:")
        print("pip install qiskit==2.2.0 qiskit-aer qiskit-machine-learning qiskit-algorithms")
        print("pip install torch torchvision tensorflow matplotlib scikit-learn")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please check your environment setup.")